import container_scrape_clean_agg_org
container_scrape_clean_agg_org.ContainerScrapeCleanAggOrg([True,True])


In [1]:
import container_scrape_clean_agg_org
container_scrape_clean_agg_org.ContainerScrapeCleanAggOrg()

containerClean_conStore.json written.
containerClean_ikea.json written.
container_agg.json written
container_org.json written.


In [13]:
my_c = container_search.ContainerSearch()

In [18]:
values = [7.,8.,5.]

pf = my_c.the_perfect_fit_values(values,2.0)
pf[0][3]


[[{'dimensions': ['7" x 9" x 5" h'],
   'dimensions text': ['7" x 9" x 5" h'],
   'image': ['https://images.containerstore.com//catalogimages/287470/10069447DeskOrganizer2Drawer_1200.jpg'],
   'new dimensions': [[7.0, 9.0, 5.0]],
   'price': ['21.99'],
   'title': '2-Drawer Desk Organizer',
   'url': 'https://www.containerstore.com/s/storage/storage-drawers/2-drawer-desk-organizer/12d?productId=11003869'},
  0],
 [{'dimensions': ['7" sq. x 5" h'],
   'dimensions text': ['7" sq. x 5" h'],
   'image': ['https://images.containerstore.com//catalogimages/246436/10065118XsmallMarcheBasketZinc_600.jpg'],
   'new dimensions': [[7.0, 7.0, 5.0]],
   'price': ['7.99'],
   'title': 'Zinc X-Small Marché Wire Storage Basket',
   'url': 'https://www.containerstore.com/s/storage/decorative-bins-baskets/zinc-x-small-marche-wire-storage-basket/12d?productId=11001002'},
  0],
 [{'dimensions': ['7" sq. x 5" h'],
   'dimensions text': ['7" sq. x 5" h'],
   'image': ['https://images.containerstore.com//cata

In [1]:
import json
import container_scrape
import container_utils
import container_search
from fractions import Fraction
import unicodedata
from pprint import pprint

In [2]:
# container_scrape.ContainerScrape_ikea()
# container_scrape.ContainerScrape_conStore()

In [3]:
my_c_conStore = container_utils.ContainerClean_conStore()
my_c_ikea=container_utils.ContainerClean_ikea()

In [4]:
my_c_conStore.create_new_dimensions()
# pprint(my_c_conStore.data[1])

In [5]:
my_c_ikea.create_new_dimensions()
# pprint(my_c_ikea.data[1])

In [6]:
my_c_conStore.write_file()
my_c_ikea.write_file()

containerClean_conStore.json written.
containerClean_ikea.json written.


In [7]:
for i in range(0,len(my_c_conStore.data)):
    d = my_c_conStore.data[i]
    if d['title'] == 'Loft Woven Storage Bins with Handles':#'Bigso Pink Milo Toy Box Seat with Handles':
        print('\n', i, '\n', d)


 393 
 {'url': 'https://www.containerstore.com/s/storage/decorative-bins-baskets/loft-woven-storage-bins-with-handles/12d?productId=11000934', 'title': 'Loft Woven Storage Bins with Handles', 'dimensions': ['16" x 11" x 9" h', '20" x 14" x 10-3/4" h'], 'price': ['21.99', '29.99'], 'image': ['https://images.containerstore.com//catalogimages/244824/10065195gCreamGreyBin_600.jpg'], 'new dimensions': [[16.0, 11.0, 9.0], [20.0, 14.0, 10.75]], 'dimensions text': ['16" x 11" x 9" h', '20" x 14" x 10-3/4" h']}


In [8]:
my_c_conStore.data[276] in my_c_conStore.data

True

In [9]:
my_c_conStore.data[276]['title'] == my_c_conStore.data[620]['title']

True

In [6]:
print(len(my_c_conStore.data))
print(len(my_c_ikea.data))

708
103


In [8]:
my_agg = container_utils.Container_Aggregate()

In [9]:
my_agg.data_conStore[1:3]

[{'dimensions': ['32-1/2" x 13-1/2" x 3-1/4" h'],
  'image': ['https://images.containerstore.com//catalogimages/331494/10019492-portable-gift-wrap-organize.jpg'],
  'new dimensions': [[32.5, 13.5, 3.25]],
  'price': ['12.99'],
  'title': 'Portable Gift Wrap Organizer',
  'url': 'https://www.containerstore.com/s/storage/holiday-storage/portable-gift-wrap-organizer/12d?productId=10010930'},
 {'dimensions': ['33-15/16" x 12" x 6-5/16" h'],
  'image': ['https://images.containerstore.com//catalogimages/269515/GiftWrapBoxClear_x.jpg'],
  'new dimensions': [[33.9375, 12.0, 6.3125]],
  'price': ['13.99'],
  'title': 'Clear Gift Wrap Box',
  'url': 'https://www.containerstore.com/s/storage/holiday-storage/clear-gift-wrap-box/12d?productId=10023588'}]

In [10]:
my_org= container_utils.Container_Organize()

In [11]:
len(my_org.data)

809

In [12]:
my_org.data[2]

{'dimensions': ['33-15/16" x 12" x 6-5/16" h'],
 'image': ['https://images.containerstore.com//catalogimages/269515/GiftWrapBoxClear_x.jpg'],
 'new dimensions': [[33.9375, 12.0, 6.3125]],
 'price': ['13.99'],
 'title': 'Clear Gift Wrap Box',
 'url': 'https://www.containerstore.com/s/storage/holiday-storage/clear-gift-wrap-box/12d?productId=10023588'}

In [13]:
my_org.organize_new_dimensions()


In [14]:
my_org.dimensions[3][5]

[{'dimensions': ['12-1/4" diam. x 6-1/2" h'],
  'image': ['https://images.containerstore.com//catalogimages/179352/10062050RoundPlateStorageBrown_l.jpg'],
  'new dimensions': [[12.25, 12.25, 6.5]],
  'price': ['39.99'],
  'title': 'Brown Twill China Storage Cases',
  'url': 'https://www.containerstore.com/s/storage/holiday-storage/brown-twill-china-storage-cases/12d?productId=10036583'},
 0]

In [15]:
my_org.write_file()

In [16]:
my_search = container_search.ContainerSearch()

In [17]:
my_search.dimensions[3][21]

[{'dimensions': ['6-3/4" x 18-1/4" x 8-3/8" h',
   '10" x 18-1/4" x 8-3/8" h',
   '13-3/8" x 18-1/4" x 8-3/8" h',
   '6-3/4" x 18-1/4" x 11" h',
   '10" x 18-1/4" x 11" h',
   '13-3/8" x 18-1/4" x 11" h'],
  'image': ['https://images.containerstore.com//catalogimages/165128/ModularDrawersSmoke100597G_2_l.jpg'],
  'new dimensions': [[6.75, 18.25, 8.375],
   [10.0, 18.25, 8.375],
   [13.375, 18.25, 8.375],
   [6.75, 18.25, 11.0],
   [10.0, 18.25, 11.0],
   [13.375, 18.25, 11.0]],
  'price': ['19.99', '24.99', '29.99', '24.99', '29.99', '34.99'],
  'title': 'Like-it Smoke Modular Drawers',
  'url': 'https://www.containerstore.com/s/storage/storage-drawers/like-it-smoke-modular-drawers/12d?productId=10034766'},
 2]

In [18]:
d = my_search.dimensions[3][21]
print(d[1])
print(d,'\n', d[0]['new dimensions'][d[1]][0])

2
[{'url': 'https://www.containerstore.com/s/storage/storage-drawers/like-it-smoke-modular-drawers/12d?productId=10034766', 'title': 'Like-it Smoke Modular Drawers', 'dimensions': ['6-3/4" x 18-1/4" x 8-3/8" h', '10" x 18-1/4" x 8-3/8" h', '13-3/8" x 18-1/4" x 8-3/8" h', '6-3/4" x 18-1/4" x 11" h', '10" x 18-1/4" x 11" h', '13-3/8" x 18-1/4" x 11" h'], 'price': ['19.99', '24.99', '29.99', '24.99', '29.99', '34.99'], 'image': ['https://images.containerstore.com//catalogimages/165128/ModularDrawersSmoke100597G_2_l.jpg'], 'new dimensions': [[6.75, 18.25, 8.375], [10.0, 18.25, 8.375], [13.375, 18.25, 8.375], [6.75, 18.25, 11.0], [10.0, 18.25, 11.0], [13.375, 18.25, 11.0]]}, 2] 
 13.375


In [33]:
pf = my_search.the_perfect_fit_values([7.,7.,7.],1.)
possibles = pf[0][3]
print(pf[1])
print(len(possibles),'\n')
for p in possibles:
    pprint(p)
    pprint(len(p[0]['price']))
    print(p[1])
    print(len(p[0]['price'])>=p[1])
    print(p[0]['price'][p[1]])

1.0999999999999999
1 

[{'dimensions': ['6-1/4" diam. x 7-1/4" h', '9-1/4" diam. x 10-1/4" h'],
  'image': ['https://images.containerstore.com//catalogimages/335458/10073573g-LEGO-storage-head.jpg'],
  'new dimensions': [[6.25, 6.25, 7.25], [9.25, 9.25, 10.25]],
  'price': ['14.99', '24.99'],
  'title': 'Lego Storage Heads',
  'url': 'https://www.containerstore.com/s/storage/lego-storage/lego-storage-heads/12d?productId=11007416'},
 0]
2
0
True
14.99
